In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.brain.internal.models as fbm
import numpy as np

model = fbm.load_model("simple-resnet-cifar10")

def find_unique(dataset_dir,sub_thresh=0.01,subset_files=None):
	try:
		fo.core.dataset.delete_non_persistent_datasets()
	except:
		pass
	dataset = fo.Dataset.from_dir(dataset_dir=dataset_dir,dataset_type=fo.types.ImageDirectory,name='temp1',)
	if subset_files is not None:
		for sample in dataset:
			if sample.filename not in subset_files:
				dataset.delete_samples(sample.id)
	embeddings = dataset.compute_embeddings(model, batch_size=16)
	results = fob.compute_similarity(dataset, embeddings=embeddings, brain_key="img_sim")
	uniques,threshes=[],[]
	for thresh in np.arange(0.01,1,0.01):
	    try:
	        results.find_duplicates(thresh=thresh)
	        print('Unique samples found:'+ str(len(results.unique_ids)) + ' at thresh='+str(thresh))
	        uniques.append(len(results.unique_ids))
	        threshes.append(thresh)
	        if len(uniques)>1:
	        	if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
	        		break
	    except:
	        pass
	arr=np.round(np.diff(np.flip(uniques))/len(dataset),4)>sub_thresh
	print("the values in arr:",arr)
	cont=len(uniques)-1
	found=False
	for j in range(cont,0,-1):
	    for i in range(len(arr)):
	        if i+j==len(arr)+1:
	            break
	        if arr[i:i+j].all():
	            idx=i
	            found=True
	            break
	    if found:
	    	cont=j
	    	break
	try:
		idx
	except NameError:		
		if len(uniques)>1:
			if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
				idx = len(uniques)-1
					
	thresh=np.flip(threshes)[idx]
	results.find_duplicates(thresh=thresh)
	print('\nThreshold fixed at:'+str(thresh)+' at continuity count of '+str(cont))
	print('Total Unique samples collected:', len(results.unique_ids))
	unique_fname=[sample.filename for sample in dataset if sample.id in results.unique_ids]
	dataset.delete()
	return unique_fname


def main():
	from find_unique import find_unique
	dataset_dir=r'\\glbmiaas01cf021\\BMGF\\DATA\\ARC_DATA\\main_test\\FAM-025-0004-2\\New folder'
	unique_files=find_unique(dataset_dir)

In [2]:
model

In [3]:
print(model)

In [4]:
model.summary()

AttributeError: 'TorchImageModel' object has no attribute 'summary'

In [6]:
mopip install pytorch-model-summary

SyntaxError: invalid syntax (3187389601.py, line 1)

In [15]:

import torch
from torch import torchsummary
from torchvision import models
from torchsummary import summary

ImportError: cannot import name 'torchsummary' from 'torch' (C:\Users\320199927\Anaconda3\lib\site-packages\torch\__init__.py)

In [13]:
import torch
# show input shape
print(summary(model, torch.zeros((1, 1, 28, 28)), show_input=True))

# show output shape
print(summary(model, torch.zeros((1, 1, 28, 28)), show_input=False))

# show output shape and hierarchical view of net
print(summary(model, torch.zeros((1, 1, 28, 28)), show_input=False, show_hierarchical=True))


AttributeError: 'TorchImageModel' object has no attribute 'children'